In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/train.csv')
holidays = pd.read_csv('../data/holidays_events.csv')
oil = pd.read_csv('../data/oil.csv')
stores = pd.read_csv('../data/stores.csv')
transactions = pd.read_csv('../data/transactions.csv')

### Train

In [3]:
train.isnull().sum()

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64

### Holidays

In [4]:
holidays.isnull().sum()

date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64

### Oil

In [5]:
oil.isnull().sum()

date           0
dcoilwtico    43
dtype: int64

In [6]:
# Back fill oil price since oil price does not change drastically day-to-day
oil['dcoilwtico'] = oil['dcoilwtico'].bfill()
oil.isnull().sum()

date          0
dcoilwtico    0
dtype: int64

### Stores

In [7]:
stores.isnull().sum()

store_nbr    0
city         0
state        0
type         0
cluster      0
dtype: int64

### Transactions

In [8]:
transactions.isnull().sum()

date            0
store_nbr       0
transactions    0
dtype: int64

### Convert Dates to datetime

In [9]:
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])
oil['date'] = pd.to_datetime(oil['date'])
holidays['date'] = pd.to_datetime(holidays['date'])
transactions['date'] = pd.to_datetime(transactions['date'])